<a href="https://colab.research.google.com/github/aditya-saurabh/Coursera_Capstone/blob/master/Bombay_Neighbourhood_Battle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from geopy.geocoders import Nominatim
import numpy as np
import requests
import folium
from pandas.io.json import json_normalize

In [2]:
#For four-square API call
CLIENT_ID = 'NXLQWZGXTP2WIFOLL5P3QMROZFNFEC4LE3JJCIVZGLLRNUF5' # your Foursquare ID
CLIENT_SECRET = 'H2RZXGB5WGWMMFNPXC3TXSAEETKXRQJCHJ5FBTD4SEZKRX54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NXLQWZGXTP2WIFOLL5P3QMROZFNFEC4LE3JJCIVZGLLRNUF5
CLIENT_SECRET:H2RZXGB5WGWMMFNPXC3TXSAEETKXRQJCHJ5FBTD4SEZKRX54


In [0]:
url_loc = 'https://www.mapsofindia.com/pincode/india/maharashtra/mumbai/'
neigh_data = pd.read_html(url_loc)

In [4]:
df = neigh_data[0]
type(df)

pandas.core.frame.DataFrame

In [5]:
df.shape

(183, 4)

In [6]:
df.head()

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A I staff colony,400029,Maharashtra,Mumbai
2,Aareymilk Colony,400065,Maharashtra,Mumbai
3,Agripada,400011,Maharashtra,Mumbai
4,Airport,400099,Maharashtra,Mumbai


In [7]:
df.columns = df.loc[0]
df.drop(index = 0, axis = 0, inplace = True)
df.head()

,Location,Pincode,State,District
1,A I staff colony,400029,Maharashtra,Mumbai
2,Aareymilk Colony,400065,Maharashtra,Mumbai
3,Agripada,400011,Maharashtra,Mumbai
4,Airport,400099,Maharashtra,Mumbai
5,Ambewadi,400004,Maharashtra,Mumbai


In [8]:
df.shape

(182, 4)

In [9]:
df.reset_index(inplace = True, drop = True)
df.head()

,Location,Pincode,State,District
0,A I staff colony,400029,Maharashtra,Mumbai
1,Aareymilk Colony,400065,Maharashtra,Mumbai
2,Agripada,400011,Maharashtra,Mumbai
3,Airport,400099,Maharashtra,Mumbai
4,Ambewadi,400004,Maharashtra,Mumbai


In [0]:
df.drop(labels = ['State', 'District'], axis = 1, inplace = True, )

In [11]:
df.head()

,Location,Pincode
0,A I staff colony,400029
1,Aareymilk Colony,400065
2,Agripada,400011
3,Airport,400099
4,Ambewadi,400004


In [0]:
geoloc = Nominatim(user_agent= 'four-square')
locations = geoloc.geocode('834001')

In [13]:
locations

Location(Chutia, Ranchi, Kanke, Ranchi, Jharkhand, 834001, India, (23.3687289, 85.3242805, 0.0))

In [0]:
df = df.groupby('Pincode').agg(lambda col: ', '.join(col)).reset_index(drop = False)

In [15]:
df.head()

,Pincode,Location
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta..."
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja..."
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B..."
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh..."
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c."


In [0]:
latitude = []
longitude = []
geolocation = Nominatim(user_agent= 'four-square')
for zipcode in df.Pincode:
  location = geolocation.geocode(query = {'country': 'India', 'city' : 'Mumbai', 'postalcode' : str(zipcode)})
  
  if location:
    latitude.append(location.latitude)
    longitude.append(location.longitude)
  else:
    latitude.append('NaN')
    longitude.append('NaN')
  


In [17]:
len(longitude)

67

In [18]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9388,72.8353
1,400002,"Kalbadevi, Ramwadi, S. c. court, Shroff Mahaja...",18.9388,72.8353
2,400003,"B P t colony, B.P.lane, Mandvi, Masjid, Noor B...",18.9388,72.8353
3,400004,"Ambewadi, Charni Road, Chaupati, Girgaon, Madh...",18.9388,72.8353
4,400005,"Asvini, Colaba, Holiday Camp, V.W.t.c.",18.9388,72.8353


In [19]:
df.describe()

,Pincode,Location,Latitude,Longitude
count,67,67,67.000000,67.000000
unique,67,67,37.000000,37.000000
top,400098,Borvali West,18.938771,72.835335
freq,1,1,28.000000,28.000000


In [0]:
df = df[df['Latitude'] != 'NaN']

In [21]:
df.describe()

,Pincode,Location,Latitude,Longitude
count,63,63,63.000000,63.000000
unique,63,63,36.000000,36.000000
top,400098,Borvali West,18.938771,72.835335
freq,1,1,28.000000,28.000000


In [0]:
df.reset_index(drop = True, inplace = True)

In [0]:
#Removing dublicate co-ordinates and replacing it with the first value
bool_data = df.duplicated(subset  = 'Latitude', keep = 'first')

In [0]:
df = df[~bool_data]

In [25]:
df.reset_index(inplace = True, drop = True)
df.head()

,Pincode,Location,Latitude,Longitude
0,400001,"Bazargate, M.P.t., Mumbai., Stock Exchange, Ta...",18.9388,72.8353
1,400011,"Agripada, Chinchpokli, Haines Road, Jacob Circle",19.0359,72.8915
2,400017,"Dharavi, Dharavi Road",19.0458,72.8554
3,400029,"A I staff colony, Santacruz P&t colony",19.078,72.8648
4,400049,Juhu,19.1092,72.8343


In [26]:
df.shape

(36, 4)

In [27]:
#Plotting the various localitites in Mumbai
bombay = folium.Map(zoom_start = 10, location= [19.07, 72.87], height = '50%')

for loc, lat, lon in zip(df.Location, df.Latitude, df.Longitude):
  folium.CircleMarker(
      [float(lat), float(lon)],
      radius = 10,
      popup = str(loc),
      color = 'red',
      fill_color = True
  ).add_to(bombay)

bombay

In [0]:
data = df.copy()

In [0]:
latitude = data['Latitude'][0]
longitude = data['Longitude'][0]

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
  CLIENT_ID,
  CLIENT_SECRET,
  VERSION, latitude, longitude,
  radius, LIMIT)
  
results = requests.get(url).json()

In [0]:
results = results['response']['groups'][0]['items']

In [0]:
result = json_normalize(results)

In [32]:
result

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,e-0-5263e1ba11d265711e8024bf-0,0,"[{'summary': 'This spot is popular', 'type': '...",5263e1ba11d265711e8024bf,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Fort,18.938550,72.833464,"[{'label': 'display', 'lat': 18.93854995852303...",198,400001,IN,Mumbai,Mahārāshtra,India,"[# 16, Near Sterling Cinema, Murzban Road, Aza...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",0,[],NaN
1,e-0-4caa0096d971b1f7ccca23e1-1,0,"[{'summary': 'This spot is popular', 'type': '...",4caa0096d971b1f7ccca23e1,Royal China,Waudby Road,NaN,18.938715,72.832933,"[{'label': 'display', 'lat': 18.93871523915629...",253,NaN,IN,Mumbai,Mahārāshtra,India,"[Waudby Road, Mumbai, Mahārāshtra, India]","[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",0,[],NaN
2,e-0-4b0587d9f964a52023a422e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b0587d9f964a52023a422e3,Sher-E-Punjab,Shahid Bhagat Singh Road,NaN,18.937944,72.837853,"[{'label': 'display', 'lat': 18.93794383535345...",280,400 001,IN,Mumbai,Mahārāshtra,India,"[Shahid Bhagat Singh Road, Mumbai 400 001, Mah...","[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],NaN
3,e-0-4c714f4fd7fab1f715d760c9-3,0,"[{'summary': 'This spot is popular', 'type': '...",4c714f4fd7fab1f715d760c9,Cafe Excelsior,"23/A, K Nayak Marg","Opposite New Excelsior Cinema, Fort",18.937701,72.833566,"[{'label': 'display', 'lat': 18.93770120763569...",221,400001,IN,Mumbai,Mahārāshtra,India,"[23/A, K Nayak Marg (Opposite New Excelsior Ci...","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",0,[],NaN
4,e-0-4babe4fff964a520f8d23ae3-4,0,"[{'summary': 'This spot is popular', 'type': '...",4babe4fff964a520f8d23ae3,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,D.N.Road,18.940088,72.835257,NaN,146,400001,IN,Mumbai,Mahārāshtra,India,"[Dr. Dadabhai Naoroji Road (D.N.Road), Mumbai ...","[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",0,[],32965357
5,e-0-5215d00311d235bf4fe158e9-5,0,"[{'summary': 'This spot is popular', 'type': '...",5215d00311d235bf4fe158e9,Dakshin Bar And Kitchen,NaN,NaN,18.936489,72.837490,"[{'label': 'display', 'lat': 18.93648945230135...",340,NaN,IN,Mumbai,Mahārāshtra,India,"[Mumbai, Mahārāshtra, India]","[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",0,[],NaN
6,e-0-4bb483c6737d76b0e86f3b7c-6,0,"[{'summary': 'This spot is popular', 'type': '...",4bb483c6737d76b0e86f3b7c,Sterling Cineplex,"Murzban Rd., Cinema Lane","Opp. CST, Fort",18.938296,72.833104,"[{'label': 'display', 'lat': 18.93829644125018...",240,400001,IN,Mumbai,Mahārāshtra,India,"[Murzban Rd., Cinema Lane (Opp. CST, Fort), Mu...","[{'id': '4bf58dd8d48988d180941735', 'name': 'M...",0,[],NaN
7,e-0-4bdaa0332a3a0f47ad8eabb6-7,0,"[{'summary': 'This spot is popular', 'type': '...",4bdaa0332a3a0f47ad8eabb6,Ideal Corner,Gunbow Street,Fort,18.934961,72.834050,"[{'label': 'display', 'lat': 18.93496147003577...",445,NaN,IN,Mumbai,Mahārāshtra,India,"[Gunbow Street (Fort), Mumbai, Mahārāshtra, In...","[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],NaN
8,e-0-4e33a472227111ae76912f24-8,0,"[{'summary': 'This spot is popular', 'type': '...",4e33a472227111ae76912f24,Pancham Puriwala,Opp gpo,NaN,18.938214,72.835697,"[{'label': 'display', 'lat': 18.93821427897314...",72,NaN,IN,Mumbai,Mahārāshtra,India,"[Opp gpo, Mumbai, Mahārāshtra, India]","[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",0,[],NaN
9,e-0-4b7b66b2f964a52011622fe3-9,0,"[{'summary': 'This spot is popular', 'type': '...",4b7b66b2f964a52011622fe3,McDonald's,D N Rd.,Opposite Mumbai CST railway station,18.938985,72.834504,"[{'label': 'display', 

In [0]:
features = ['venue.name', 'venue.location.address', 'venue.categories']

In [34]:
result = result[features]
result.head()

,venue.name,venue.location.address,venue.categories
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B..."
1,Royal China,Waudby Road,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C..."
2,Sher-E-Punjab,Shahid Bhagat Singh Road,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I..."
3,Cafe Excelsior,"23/A, K Nayak Marg","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C..."
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T..."


In [35]:
result.iloc[0]['venue.categories'][0]['name']

'Bar'

In [36]:
result['venue.type'] = result['venue.categories'].apply(lambda x : x[0]['name'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
result.head()

,venue.name,venue.location.address,venue.categories,venue.type
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...","[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",Bar
1,Royal China,Waudby Road,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",Chinese Restaurant
2,Sher-E-Punjab,Shahid Bhagat Singh Road,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",Indian Restaurant
3,Cafe Excelsior,"23/A, K Nayak Marg","[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",Café
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,"[{'id': '4bf58dd8d48988d129951735', 'name': 'T...",Train Station


In [38]:
result.columns

Index(['venue.name', 'venue.location.address', 'venue.categories',
       'venue.type'],
      dtype='object')

In [39]:
result.drop(labels = ['venue.categories'], inplace=True, axis=1)
result.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,venue.name,venue.location.address,venue.type
0,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Bar
1,Royal China,Waudby Road,Chinese Restaurant
2,Sher-E-Punjab,Shahid Bhagat Singh Road,Indian Restaurant
3,Cafe Excelsior,"23/A, K Nayak Marg",Café
4,Chhatrapati Shivaji Maharaj Terminus,Dr. Dadabhai Naoroji Road,Train Station


In [40]:
result.rename(columns={'venue.name':'name', 'venue.location.address':'address', 'venue.type':'type'}, inplace=True)
result.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


(19, 3)

In [0]:
columns = ['name', 'address', 'type', 'latitude', 'longitude', 'location']
data_final = pd.DataFrame(columns = columns)

def get_locations (url):
  results = requests.get(url).json()
  try:
    results = results['response']['groups'][0]['items']
    result = json_normalize(results)
    result = result[features]
    result['venue.type'] = result['venue.categories'].apply(lambda x : x[0]['name'])
    result.drop(labels = ['venue.categories'], inplace=True, axis=1)
    result.rename(columns={'venue.name':'name', 'venue.location.address':'address', 'venue.type':'type'}, inplace=True)
    return(result)

  except:
    return data_final
  
  
  


In [0]:
#Calling Four Square API

LIMIT = 150
radius = 1500

for loc, lat, lon in zip(data.Location, data.Latitude, data.Longitude):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
  CLIENT_ID,
  CLIENT_SECRET,
  VERSION, lat, lon,
  radius, LIMIT)
  
  result = get_locations(url)
  result['latitude'] = result.shape[0]*[lat]
  result['longitude'] = result.shape[0]*[lon]
  result['location'] = result.shape[0]*[loc]

  data_final = data_final.append(result, ignore_index=True )

In [64]:
len(data_final['type'].unique())

197

In [65]:
data_final.describe(include = 'all')

,name,address,type,latitude,longitude,location
count,11646,9792,11646,11646.000000,11646.000000,11646
unique,1306,1080,197,NaN,NaN,17
top,Cafe Coffee Day,Oberoi Mall,Indian Restaurant,NaN,NaN,"Aareymilk Colony, Nagari Niwara"
freq,294,117,1353,NaN,NaN,10634
mean,NaN,NaN,NaN,19.179087,72.893197,NaN
std,NaN,NaN,NaN,0.015456,0.012723,NaN
min,NaN,NaN,NaN,19.076136,72.818314,NaN
25%,NaN,NaN,NaN,19.179371,72.896980,NaN
50%,NaN,NaN,NaN,19.179371,72.896980,NaN
75%,NaN,NaN,NaN,19.179371,72.896980,NaN


In [66]:
data_final

,name,address,type,latitude,longitude,location
0,Royal China,Waudby Road,Chinese Restaurant,19.179371,72.896980,"Aareymilk Colony, Nagari Niwara"
1,Town House Cafe,"# 16, Near Sterling Cinema, Murzban Road, Azad...",Bar,19.179371,72.896980,"Aareymilk Colony, Nagari Niwara"
2,Sher-E-Punjab,Shahid Bhagat Singh Road,Indian Restaurant,19.179371,72.896980,"Aareymilk Colony, Nagari Niwara"
3,Cafe Excelsior,"23/A, K Nayak Marg",Café,19.179371,72.896980,"Aareymilk Colony, Nagari Niwara"
4,Yazdani Bakery,Cawasji Patel Street,Bakery,19.179371,72.896980,"Aareymilk Colony, Nagari Niwara"
...,...,...,...,...,...,...
11641,Oshiwara Old Furniture Market,NaN,Flea Market,19.159050,72.841371,Motilal Nagar
11642,राम मंदिर रोड स्टेशन,राम मंदिर रोड,Train Station,19.159050,72.841371,Motilal Nagar
11643,sweetish house mafia,NaN,Dessert Shop,19.159050,72.841371,Motilal Nagar
11644,station road tapri,NaN,Tea Room,19.159050,72.841371,Motilal Nagar


In [0]:
#One Hot Encoding
one_hot = pd.get_dummies(data_final['type'])

In [68]:
one_hot.head()

,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,...,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
one_hot['location'] = data_final['location']

In [0]:
neigh_bombay = one_hot

In [0]:
neigh_bombay_filtered = neigh_bombay.groupby(by = 'location').mean()

In [72]:
neigh_bombay_filtered.head()

,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,...,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Aareymilk Colony, Nagari Niwara",0.001128,0.000564,0.000376,0.000564,0.004702,0.001128,0.003950,0.000564,0.002069,0.021065,0.001881,0.000564,0.000564,0.006959,0.002257,0.021441,0.000188,0.030656,0.003385,0.000564,0.001693,0.001505,0.003950,0.006583,0.001881,0.000564,0.000564,0.006207,0.004138,0.000188,0.000752,0.004138,0.001128,0.003385,0.000188,0.049276,0.000564,0.000188,0.000752,0.028211,...,0.000752,0.020312,0.000188,0.000752,0.002257,0.000564,0.025014,0.003009,0.021817,0.000564,0.000564,0.007335,0.003197,0.016927,0.000752,0.000564,0.002069,0.005642,0.001128,0.000188,0.003573,0.000564,0.000188,0.003197,0.001128,0.004514,0.000752,0.003762,0.001128,0.000564,0.001317,0.005642,0.015798,0.000188,0.000188,0.000564,0.001128,0.000376,0.004138,0.001128
"Airport, International Airport, Sahar P & t colony, Sahargaon",0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.071429,0.010204,0.000000,0.000000,0.051020,...,0.000000,0.030612,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.040816,0.000000,0.000000,0.010204,0.010204,0.020408,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
"Andheri East, Nagardas Road",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.020408,0.020408,0.000000,0.000000,0.020408,...,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.061224,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Borivali,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.106383,0.000000,0.000000,0.000000,0.042553,...,0.000000,0.148936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.

In [73]:
neigh_bombay_filtered.shape

(17, 197)

In [0]:
# Applying K Means with four labels

In [0]:
from sklearn.cluster import KMeans

In [0]:
X = neigh_bombay_filtered

In [0]:
kmeans = KMeans(n_clusters=4, random_state=5).fit(X)

In [0]:
label = kmeans.labels_

In [0]:
#adding labels to the cluster

In [0]:
neigh_bombay_filtered['labels'] = label

In [81]:
neigh_bombay_filtered.head()

,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,...,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Track,Train,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio,labels
location,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Aareymilk Colony, Nagari Niwara",0.001128,0.000564,0.000376,0.000564,0.004702,0.001128,0.003950,0.000564,0.002069,0.021065,0.001881,0.000564,0.000564,0.006959,0.002257,0.021441,0.000188,0.030656,0.003385,0.000564,0.001693,0.001505,0.003950,0.006583,0.001881,0.000564,0.000564,0.006207,0.004138,0.000188,0.000752,0.004138,0.001128,0.003385,0.000188,0.049276,0.000564,0.000188,0.000752,0.028211,...,0.020312,0.000188,0.000752,0.002257,0.000564,0.025014,0.003009,0.021817,0.000564,0.000564,0.007335,0.003197,0.016927,0.000752,0.000564,0.002069,0.005642,0.001128,0.000188,0.003573,0.000564,0.000188,0.003197,0.001128,0.004514,0.000752,0.003762,0.001128,0.000564,0.001317,0.005642,0.015798,0.000188,0.000188,0.000564,0.001128,0.000376,0.004138,0.001128,1
"Airport, International Airport, Sahar P & t colony, Sahargaon",0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.071429,0.010204,0.000000,0.000000,0.051020,...,0.030612,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.040816,0.000000,0.000000,0.010204,0.010204,0.020408,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
"Andheri East, Nagardas Road",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.020408,0.000000,0.020408,0.020408,0.000000,0.000000,0.020408,...,0.020408,0.000000,0.000000,0.000000,0.000000,0.061224,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2
Borivali,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.106383,0.000000,0.000000,0.000000,0.042553,...,0.148936,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000

In [0]:
color = ['blue', 'yellow', 'green', 'orange']

In [83]:
neigh_bom_map = folium.Map(zoom_start = 10, location= [19.07, 72.87], height = '50%')

for loc in neigh_bombay_filtered.index:
  lat = df[df['Location'] == loc]['Latitude'].unique()[0]
  lon = df[df['Location'] == loc]['Longitude'].unique()[0]
  folium.CircleMarker(
      [float(lat), float(lon)],
      radius = 10,
      popup = str(loc),
      color = color[int(neigh_bombay_filtered.loc[loc]['labels'])],
      fill_color = True
  ).add_to(neigh_bom_map)

neigh_bom_map